In [1]:
import tensorflow as tf
tf.__version__

'1.14.0'

## TensorFlow를 이용한 선형회귀

지금까지 배운 개념을 이용해 보스턴 주택가격을 예측하는 선형회귀모형을 구현해 보자.

In [2]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['const'] = np.ones(df.shape[0])
df.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,const
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,1.0
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,1.0
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,1.0
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,1.0
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88,1.0


In [3]:
n, m = df.shape
n, m

(506, 14)

In [0]:
X = tf.placeholder(tf.float64, shape=(n, m))
y = tf.placeholder(tf.float64, shape=(n, 1))

XT = tf.transpose(X)
w = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)  # ((X.T * X)^-1 * XT) * y
y_pred = tf.matmul(X, w)

In [5]:
with tf.Session() as sess:
  y_pred = sess.run(y_pred, feed_dict={X: df.values, y:boston.target.reshape(-1, 1)})
  
print("예측한 집값:", y_pred[19], "실제 집값:", boston.target[19])

예측한 집값: [18.40613603] 실제 집값: 18.2


## 퍼셉트론

이번엔 **Iris** 데이터 중 두 종류를 분류하는 퍼셉트론을 제작한다. y값은 1 또는 -1을 사용하고 활성화 함수로는 하이퍼탄젠트(hypertangent)함수를 사용한다.

In [0]:
from sklearn.datasets import load_iris
iris = load_iris()
idx = np.in1d(iris.target, [0, 2])
X_data = iris.data[idx, 0:2]
y_data = (iris.target[idx] - 1.0)[:, np.newaxis]  # 2차원으로 만들어야 함!!!

## 최적화

TensorFlow에는 Theano와 달리 최적화를 위한 **GradientDescentOptimizer** 등의 클래스가 미리 구현되어 있으므로 사용자가 구현할 필요가 없다. 

(1) 최적화 관련 클래스를 생성하고 (2) minimize명령에 최소화 하고 싶은 값의 텐서를 입력하면 된다.

In [7]:
tf.reset_default_graph()  # 지금까지 생성된 모든 텐서를 그래프에서 제거

np.random.seed(0)
with tf.variable_scope("perceptron", reuse=tf.AUTO_REUSE):
  w = 1e-3 * tf.get_variable("weight", [2, 1], dtype=tf.float64, initializer=tf.random_uniform_initializer())  # 가중치 정의
  b = 1e-3 * tf.get_variable("bias", [], dtype=tf.float64, initializer=tf.random_uniform_initializer()) 
  z = tf.tanh(tf.matmul(X_data, w) + b)
  
zero = np.zeros(1, dtype=np.float64)[0]  # float64 자료형 0 생성
cost = tf.reduce_sum(tf.maximum(zero, tf.multiply(-y_data, z)))
optimizer = tf.train.GradientDescentOptimizer(1e-1)  # 최적화 클래스 생성
train = optimizer.minimize(cost)  # train

W0906 15:01:56.071173 140617285932928 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
